<a href="https://colab.research.google.com/github/JMarsH-MLiS/drone_project/blob/main/James%2BScott.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
from itertools import combinations

# Card Class
class Card:
    def __init__(self, colour, number):
        assert isinstance(number, int)
        self.colour = colour
        self.number = number

    def __str__(self):
        return f"{self.colour} {self.number}"

# Computer Player Class
class Computer_player:
    def __init__(self, card_strings):
        print("Hand at start of turn:", card_strings)
        self.collection = [Card(colour, int(number)) for colour, number in (card.split() for card in card_strings)]

    def __str__(self):
        return ', '.join(str(card) for card in self.collection)

    def find_largest_valid_ofakind(self, available_cards):
        for n in range(len(available_cards), 0, -1):
            for sub_set in combinations(available_cards, n):
                if self.check_valid_ofakind(sub_set):
                    return list(sub_set)
        return None

    def find_largest_valid_straight(self, available_cards):
        sorted_collection = sorted(available_cards, key=lambda card: card.number)
        for n in range(len(available_cards), 0, -1):
            for sub_set in combinations(sorted_collection, n):
                if self.check_valid_straight(sub_set):
                    return list(sub_set)
        return None

    def check_valid_ofakind(self, group):
        if not group or len(group) < 3:
            return False
        colours = {card.colour for card in group}
        numbers = {card.number for card in group}
        return len(numbers) == 1 and len(colours) == len(group)

    def check_valid_straight(self, group):
        if not group or len(group) < 3:
            return False
        colours = {card.colour for card in group}
        numbers = sorted(card.number for card in group)
        return len(colours) == 1 and (max(numbers) - min(numbers) == len(numbers) - 1)

    def find_multiple_valid_groups(self, straight_first=True):
        available_cards = self.collection.copy()
        valid_groups = []
        while len(available_cards) >= 3:
            found_group = False
            if straight_first:
                straight_group = self.find_largest_valid_straight(available_cards)
                if straight_group:
                    valid_groups.append([str(card) for card in straight_group])
                    for card in straight_group:
                        available_cards.remove(card)
                    found_group = True
            ofakind_group = self.find_largest_valid_ofakind(available_cards)
            if ofakind_group:
                valid_groups.append([str(card) for card in ofakind_group])
                for card in ofakind_group:
                    available_cards.remove(card)
                found_group = True
            if not found_group:
                break
        return valid_groups, available_cards

    def combine_missing_cards(self):
        missing_cards = self.find_missing_three_of_a_kind() + self.find_missing_straight_flush()
        return missing_cards

    def find_missing_three_of_a_kind(self):
        missing_cards = []
        number_to_cards = {}
        for card in self.collection:
            number_to_cards.setdefault(card.number, []).append(card)
        for number, cards in number_to_cards.items():
            if len(cards) == 2:
                all_colours = {"red", "blue", "green", "yellow"}
                card_colours = {card.colour for card in cards}
                missing_colours = all_colours - card_colours
                for colour in missing_colours:
                    missing_cards.append(Card(colour, number))
        return missing_cards

    def find_missing_straight_flush(self):
        missing_cards = []
        colour_to_cards = {colour: [] for colour in {"red", "blue", "green", "yellow"}}
        for card in self.collection:
            colour_to_cards[card.colour].append(card)
        for colour, cards in colour_to_cards.items():
            sorted_cards = sorted(cards, key=lambda card: card.number)
            for i in range(len(sorted_cards) - 1):
                current, next_card = sorted_cards[i], sorted_cards[i + 1]
                if next_card.number - current.number == 2:
                    missing_cards.append(Card(colour, current.number + 1))
        return missing_cards

    def steal_or_draw(self, player_1, player_2, combined_missing_cards, new_hand):
        initial_deck = [f"{colour} {num}" for colour in ["red", "green", "blue", "yellow"] for num in range(1, 11)] * 2
        current_deck = initial_deck.copy()
        for card_str in player_1 + player_2 + [str(card) for card in new_hand]:
            current_deck.remove(card_str)
        locations = [player_1, player_2, current_deck]
        location_names = ["Player 1", "Player 2", "Deck"]
        scores = []
        for location in locations:
            score = sum(location.count(str(card)) / len(location) for card in combined_missing_cards if location)
            scores.append(score)
        best_location_index = scores.index(max(scores))
        source = locations[best_location_index]
        if source:
            drawn_card = random.choice(source)
            new_hand.append(Card(*drawn_card.split()))
            source.remove(drawn_card)
            print(f"Drew card {drawn_card} from {location_names[best_location_index]}")

    def run(self, player_1, player_2):
        valid_groups_1, leftovers_1 = self.find_multiple_valid_groups(straight_first=False)
        valid_groups_2, leftovers_2 = self.find_multiple_valid_groups(straight_first=True)
        self.collection = leftovers_1 if len(leftovers_1) <= len(leftovers_2) else leftovers_2
        combined_missing = self.combine_missing_cards()
        self.steal_or_draw(player_1, player_2, combined_missing, self.collection)

# Example setup
collection = ["yellow 4", "green 4", "blue 4"]
player_1 = ["yellow 5", "yellow 10", "red 8", "blue 9", "blue 2"]
player_2 = []

computer_player = Computer_player(collection)
computer_player.run(player_1, player_2)


Hand at start of turn: ['yellow 4', 'green 4', 'blue 4']


AssertionError: 